In [10]:
# Standard library imports
import sqlite3

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Display plots inline
%matplotlib inline

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

# Connect to SQLite database
conn = sqlite3.connect('phishing.db')

# List all tables in the database
query_tables = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query_tables, conn)
print("Available tables in the database:")
print(tables)

# Load the main dataset (assuming the table name is 'phishing_data')
table_name = 'phishing_data'  # Replace with actual table name from above output
df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

# Close database connection
conn.close()

# Display basic dataset information
print(f"\nDataset shape: {df.shape}")
print("\nFirst 5 rows of the dataset:")
df.info()
print(df['label'].value_counts(normalize=True))
df.head()


Available tables in the database:
            name
0  phishing_data

Dataset shape: (10500, 16)

First 5 rows of the dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10500 entries, 0 to 10499
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         10500 non-null  int64  
 1   LineOfCode         8145 non-null   float64
 2   LargestLineLength  10500 non-null  int64  
 3   NoOfURLRedirect    10500 non-null  int64  
 4   NoOfSelfRedirect   10500 non-null  int64  
 5   NoOfPopup          10500 non-null  int64  
 6   NoOfiFrame         10500 non-null  int64  
 7   NoOfImage          10500 non-null  int64  
 8   NoOfSelfRef        10500 non-null  int64  
 9   NoOfExternalRef    10500 non-null  int64  
 10  Robots             10500 non-null  int64  
 11  IsResponsive       10500 non-null  int64  
 12  Industry           10500 non-null  object 
 13  HostingProvider    10500 non-null  objec

,Unnamed: 0,LineOfCode,LargestLineLength,NoOfURLRedirect,NoOfSelfRedirect,NoOfPopup,NoOfiFrame,NoOfImage,NoOfSelfRef,NoOfExternalRef,Robots,IsResponsive,Industry,HostingProvider,DomainAgeMonths,label
0,0,NaN,9965,0,0,2,1,48,172,170,1,1,Fashion,DreamHost,98,0
1,1,769.0,9507,0,0,0,4,25,55,78,1,1,Fashion,AWS,31,1
2,2,720.0,2550,0,0,0,2,-31,94,13,0,1,Education,Google Cloud,26,1
3,3,198.0,869,0,0,0,0,0,0,2,0,0,Unknown,DreamHost,2,1
4,4,972.0,2807,0,0,0,2,6,83,51,0,1,Education,GoDaddy,43,1


# The purpose of the step: Data Acquisition

The initial phase involves establishing a connection to the SQLite database and retrieving the phishing website dataset. We first inspect the database structure to identify available tables, then load the relevant data into a pandas DataFrame for subsequent analysis. This structured approach ensures data integrity and provides a foundation for all subsequent analytical steps.

The dataset is now loaded and ready for comprehensive exploratory analysis.

# The conclusions and the interpretation

## Data Quality Assessment and Initial Findings
Data Structure Overview
The dataset comprises 10,500 total entries with 16 features, including both numerical and categorical variables. The data types are appropriately assigned, with integer and float types for numerical features and object type for the categorical Industry variable. The target variable is also balanced since there are 55% of websites are being labelled as legitimate and 45% of websites are being labelled as phishing website.

## Critical Data Quality Issues
Missing Values Identification:

LineOfCode contains 2,355 missing values (22.4% of total records)

All other features demonstrate complete data integrity with 10,500 non-null entries

Data Completeness Impact:
The significant missingness in LineOfCode presents a substantial data quality concern. This feature's partial availability may limit its utility in subsequent analysis and modeling phases. The handling strategy for these missing values will require careful consideration, potentially involving imputation techniques or exclusion from certain analyses.

## Preliminary Statistical Interpretation
Dataset Scale and Scope:
With 10,500 observations, the dataset provides a substantial foundation for robust statistical analysis and machine learning model development. The sample size is adequate for training complex models while maintaining generalization capabilities.

Feature Diversity:
The inclusion of both technical metrics (LineOfCode, LargestLineLength, NoOfURLRedirect) and categorical attributes (Industry, Robots, IsResponsive) enables comprehensive feature engineering opportunities. This diversity supports the development of a multi-faceted phishing detection approach.

## Implications for Analysis Strategy
Data Preprocessing Requirements:
The identified data quality issues necessitate implementing rigorous preprocessing pipelines. Specific considerations include:

Development of appropriate missing data handling strategies for LineOfCode

Validation of data distributions for numerical features to identify potential outliers

Encoding schemes for categorical variables like Industry and binary features

In [11]:
# Analyze missing data pattern
missing_pattern = df[df['LineOfCode'].isnull()]
complete_pattern = df[df['LineOfCode'].notnull()]

print("Missing LineOfCode statistics:")
print(f"Total missing: {len(missing_pattern)}")
print(f"Percentage missing: {(len(missing_pattern)/len(df))*100:.1f}%")

# Check if missingness is related to other variables or random
print("\nMissing value pattern analysis:")
for column in ['label', 'Industry', 'IsResponsive']:
    missing_dist = missing_pattern[column].value_counts(normalize=True)
    complete_dist = complete_pattern[column].value_counts(normalize=True)
    print(f"\n{column} distribution comparison:")
    print("Missing data: ", missing_dist.head())
    print("Complete data: ", complete_dist.head())

Missing LineOfCode statistics:
Total missing: 2355
Percentage missing: 22.4%

Missing value pattern analysis:

label distribution comparison:
Missing data:  label
0    1.0
Name: proportion, dtype: float64
Complete data:  label
1    0.709515
0    0.290485
Name: proportion, dtype: float64

Industry distribution comparison:
Missing data:  Industry
eCommerce     0.286624
Unknown       0.170701
Education     0.146497
Non-profit    0.097665
Fashion       0.062420
Name: proportion, dtype: float64
Complete data:  Industry
eCommerce     0.226642
Non-profit    0.165746
Education     0.140086
Unknown       0.128177
Fashion       0.084960
Name: proportion, dtype: float64

IsResponsive distribution comparison:
Missing data:  IsResponsive
0    0.562633
1    0.437367
Name: proportion, dtype: float64
Complete data:  IsResponsive
1    0.677471
0    0.322529
Name: proportion, dtype: float64


# The Purpose of the step: Missing Data Assessment
The LineOfCode feature contains 2,355 missing values, representing approximately 22.4% of the total dataset. Before implementing any handling strategy, it is crucial to understand the nature and pattern of these missing values.

# The conclusion and implication
Non-Random Missing Data Pattern
Based on the analysis results, we have identified a highly significant pattern in the missing data that requires specialized handling.

### Missing Data Pattern Analysis
The analysis reveals a critical finding:
All missing LineOfCode values occur exclusively in phishing websites (label = 0)

### Analytical Implications
Missingness as a Predictive Feature
The fact that missing LineOfCode data occurs exclusively in phishing websites transforms this data quality issue into a potentially powerful predictive feature. This pattern suggests:

Phishing websites may intentionally obfuscate or fail to provide code metrics

Technical limitations in data collection from malicious sites

Systematic differences in how phishing vs legitimate websites are structured

It is possible to create enhanced features that leverage this pattern for Feature Engineering Opportunity.

### Strategy

1. Primary Approach: Use the class-specific median imputation with pattern preservation

2. Feature Inclusion: Retain both the imputed LineOfCode values and the missing indicator features

3. Model Interpretation: Document that missing LineOfCode is a strong indicator of phishing websites

4. Risk Assessment: Acknowledge that this pattern may reflect underlying data collection biases that should be considered in model deployment



In [ ]:
def handle_missing_lineofcode_strategic(df):
    """
    Handle missing LineOfCode using class-specific imputation
    that preserves the missing data pattern as a feature
    """
    df_processed = df.copy()
    
    # Create comprehensive missing indicator
    df_processed['LineOfCode_Missing'] = df_processed['LineOfCode'].isnull().astype(int)
    
    # Calculate separate medians for each class
    median_phishing = df_processed[df_processed['label'] == 0]['LineOfCode'].median()
    median_legitimate = df_processed[df_processed['label'] == 1]['LineOfCode'].median()
    
    print(f"Median LineOfCode - Phishing sites: {median_phishing}")
    print(f"Median LineOfCode - Legitimate sites: {median_legitimate}")
    
    # Impute missing values with class-specific medians
    phishing_mask = (df_processed['LineOfCode'].isnull()) & (df_processed['label'] == 0)
    legitimate_mask = (df_processed['LineOfCode'].isnull()) & (df_processed['label'] == 1)
    
    df_processed.loc[phishing_mask, 'LineOfCode'] = median_phishing
    df_processed.loc[legitimate_mask, 'LineOfCode'] = median_legitimate
    
    # Create interaction feature that captures the pattern
    df_processed['LineOfCode_Missing_Phishing'] = (
        (df_processed['LineOfCode_Missing'] == 1) & 
        (df_processed['label'] == 0)
    ).astype(int)
    
    return df_processed

# Apply strategic imputation
df_final = handle_missing_lineofcode_strategic(df)

# Validate the imputation
print("\n=== IMPUTATION VALIDATION ===")
print(f"Original missing values: 2355")
print(f"Remaining missing values: {df_final['LineOfCode'].isnull().sum()}")
print(f"Missing indicator distribution:")
print(df_final['LineOfCode_Missing'].value_counts())
# print(f"Pattern feature distribution:")
# print(df_final['LineOfCode_Missing_Phishing'].value_counts())

# # Assess impact on class distributions
# print("\n=== CLASS DISTRIBUTION ANALYSIS ===")
# print("LineOfCode_Missing by label:")
# print(pd.crosstab(df_final['LineOfCode_Missing'], df_final['label']))

Median LineOfCode - Phishing sites: 37.0
Median LineOfCode - Legitimate sites: 888.0

=== IMPUTATION VALIDATION ===
Original missing values: 2355
Remaining missing values: 0
Missing indicator distribution:
LineOfCode_Missing
0    8145
1    2355
Name: count, dtype: int64
Pattern feature distribution:
LineOfCode_Missing_Phishing
0    8145
1    2355
Name: count, dtype: int64

=== CLASS DISTRIBUTION ANALYSIS ===
LineOfCode_Missing by label:
label                  0     1
LineOfCode_Missing            
0                   2366  5779
1                   2355     0
